In [4]:
import os
import pickle
from glob import iglob
import numpy as np
import librosa

# if wavs for testing are put in 'wavs' folder then:
# TEST_DATA_AUDIO_DIR = './for_user/wavs'

TEST_DATA_AUDIO_DIR = ''
CONVERTED_DATA_DIR = './pkls'
TARGET_SR = 8000
AUDIO_LENGTH = 10000

def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

def convert_testig_data():
    for i, wav_filename in enumerate(iglob(os.path.join(TEST_DATA_AUDIO_DIR, '**.wav'), recursive=True)):
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            print('PAD New length =', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
            audio_buf = audio_buf[0:AUDIO_LENGTH]
            print('CUT New length =', len(audio_buf))

        output_folder = CONVERTED_DATA_DIR
        output_filename = os.path.join(output_folder, str(i) + '.pkl')
        file_name = os.path.splitext(os.path.basename(wav_filename))[0]
        out = {'audio': audio_buf,
               'sr': TARGET_SR,
                'filename': file_name}
        with open(output_filename, 'wb') as w:
            pickle.dump(out, w)


if __name__ == '__main__':
    convert_testig_data()

In [ ]:
MODEL_FILENAME = 'best-trained-model.hdf5'
from glob import glob
import pandas as pd

def get_data(file_list):
    def load_into(_filename, _x, _y):
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            _x.append(audio_element['audio'])
            _y.append(audio_element['filename'])
    x = []
    y = []
    for filename in file_list:
        load_into(filename, x, y)
    return np.array(x), np.array(y)

files = glob(os.path.join(CONVERTED_DATA_DIR, '**.pkl'))
x, indexes = get_data(files)

In [ ]:
x

In [ ]:
indexes

In [ ]:
from keras.models import load_model

# model = pickle.load(open(MODEL_FILENAME, 'rb'))
model = load_model(MODEL_FILENAME)
# x_n = x.reshape(x.shape[0], 40, 250, 1)
prediction = model.predict(x)
prediction = pd.DataFrame(prediction)
prediction

In [ ]:
prediction.shape

In [ ]:
indexes = pd.DataFrame(indexes)
indexes

In [ ]:
result = pd.concat([ pd.DataFrame(indexes),  pd.DataFrame(prediction)], axis=1)
result

In [ ]:
np.array(result)

In [ ]:
# np.savetxt("predictions.csv", np.array(result))
# result.to_csv("predictions.csv")
result.to_csv('13-submission.csv', index=False)

In [ ]:
check = pd.read_csv('13-submission.csv')
check